# Boundary layer models

The interaction of the primary vortex with the Earth's surface is where we want to focus for a hazard model. This is where the effects of a TC are felt, so we want to capture the wind speeds at the surface. 



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm as cmap
from inspect import getargspec
import numpy as np

from wind import windmodels
from Utilities import metutils
from Utilities.maputils import bearing2theta, makeGrid, meshLatLon, gridLatLonDist
from PlotInterface.maps import saveWindfieldMap

from ipywidgets import interact, fixed, FloatSlider, Dropdown
import ipywidgets as widgets

import seaborn as sns
sns.set_style('ticks', {'image.cmap':'coolwarm'})
sns.set_context('poster')

Some simple functions to help with setting up the wind fields.

In [ ]:
def polarGridAroundEye(lon, lat, margin=3, resolution=0.01):
    R, theta = makeGrid(lon, lat, margin, resolution)
    return R, theta

def meshGrid(lon, lat, margin=3, resolution=0.01):
    xgrid, ygrid = meshLatLon(lon, lat, margin, resolution)
    return xgrid, ygrid

Here we call the functions from TCRM to generate a surface wind field that incorporates the effects of storm forward motion and surface friction. There are two primary steps:
1. Create a radial profile that represents the primary vortex at the gradient level. This is the `cls = windmodels.profile(profileType)` line.
    1. If the selected profile is the "Holland" profile, we need to supply the $\beta$ parameter value to the method call that calculates the profile (`profile = cls(...)`)
    2. If the selected profile is the "DoubleHolland" profile, we need to supply the $\beta$ parameter value to the method call twice, once for each of the components of the profile. They can be different, but here we leave them the same.
    3. Otherwise, this parameter is calculated automatically, or not required.
2. Calculate the surface wind field, using the chosen radial profile: `windfield = cls(profile)`
We then determine the eastward and northward components of the wind field (`Ux`, `Vy`).

In [ ]:
def localWindField(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                   profileType='powell', windFieldType='kepert'):
    thetaMax = 70.
    cls = windmodels.profile(profileType)
    if profileType=="holland":
        profile = cls(lat, lon, pEnv, pCentre, rMax*1000, beta)
    else:
        profile = cls(lat, lon, pEnv, pCentre, rMax*1000)
    R, theta = polarGridAroundEye(lon, lat)
    
    cls = windmodels.field(windFieldType)
    windfield = cls(profile)
    
    Ux, Vy = windfield.field(R*1000, theta, vFm, thetaFm, thetaMax)
    return Ux, Vy

This next function is a plotting routine. Given the parameters of a cyclone, calculate the surface wind field and plot up the various components.

In [ ]:
def plotWindfield(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                  profileType='powell', windFieldType='kepert'):
    pCentre *= 100.
    pEnv *= 100.
    vFm /= 3.6
    thetaFm = bearing2theta(np.pi * thetaFm / 180.)

    xgrid, ygrid = meshGrid(lon, lat)
    
    Ux, Vy = localWindField(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                            profileType=profileType, windFieldType=windFieldType)
    cx = rMax*np.sin(np.arange(0.00,6.29,0.01))  # Circle for rmw plots.
    cy = rMax*np.cos(np.arange(0.00,6.29,0.01))
    xlims=[lon-2, lon+2]
    ylims=[lat-2, lat+2]
    xkmlims = [-150, 150]
    ykmlims = [-150, 150]
    
    fig, ax = plt.subplots(1, 1, subplot_kw={'aspect':'equal'}, figsize=(12, 12))
    ax.hold(True)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax.contourf(xgrid, ygrid, np.sqrt(Ux*Ux+Vy*Vy), np.arange(0., 101, 5))
    ax.streamplot(xgrid, ygrid, Ux, Vy, density=3, color='k', linewidth=1)
    plt.colorbar(cm, ax=ax, orientation='horizontal')
    
    vmax = np.max(np.sqrt(Ux*Ux+Vy*Vy))*3.6
    ax.set_title("Maximum wind speed: {0:.1f} km/h".format(vmax))
    fig.tight_layout()
    plt.show()

Now we create a list of the available profiles and boundary layer models. This list is built into the `windmodels` class to enable this kind of introspection. The lists will be used in a dropdown list to allow you to switch the profile or boundary layer model in the interactive plotting routine. 

In [ ]:
profileOpts = windmodels.PROFILES.keys()
blOpts = windmodels.FIELDS.keys()

In [ ]:
interact(plotWindfield, 
         lon=fixed(120),
         lat=FloatSlider(value=-20., min=-30., max=0, step=0.1, description="Latitude"), 
         pEnv=FloatSlider(value=995, min=995, max=1000, step=1, description="Environmental pressure (Pa)"),
         pCentre=FloatSlider(value=950, min=900, max=990, step=1, description="Central pressure (Pa)"),
         rMax=FloatSlider(value=25, min=10, max=100, step=1, description="Radius to maximum wind (km)"),
         vFm=FloatSlider(value=10, min=0, max=30, step=0.1, description="Forward speed (km/h)"),
         thetaFm=FloatSlider(value=180, min=0, max=360, step=1, description="Storm bearing"),
         beta=FloatSlider(value=1.6, min=1.2, max=2.5, step=0.05, description="Beta parameter"),
         profileType=Dropdown(options=profileOpts, value="powell", description="Radial profile"),
         windFieldType=Dropdown(options=blOpts, value="kepert", description="Boundary layer model"))

In [ ]:

def _plotWindfield(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                  profileType='powell', windFieldType='kepert'):
    thetaFm = thetaFm * np.pi / 180.
    xgrid, ygrid = meshGrid(lon, lat)
    
    Ux, Vy = localWindField(lon, lat, pEnv, pCentre, rMax, vFm, thetaFm, beta, 
                            profileType=profileType, windFieldType=windFieldType)
    cx = rMax*np.sin(np.arange(0.00,6.29,0.01))  # Circle for rmw plots.
    cy = rMax*np.cos(np.arange(0.00,6.29,0.01))
    
    xlims=[lon-2, lon+2]
    ylims=[lat-2, lat+2]
    xkmlims = [-150, 150]
    ykmlims = [-150, 150]
    fig, axes = plt.subplots(2, 2,subplot_kw={'aspect':'equal'}, figsize=(16, 16))
    ax = axes.flatten()
    ax[0].hold(True)
    ax[0].set_xlim(xlims)
    ax[0].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[0].contourf(xgrid, ygrid, Ux, np.arange(-100., 101, 5))
    cs = ax[0].contour(xgrid, ygrid, Ux, np.arange(-100, 101, 5), colors='k')
    ax[0].clabel(cs, fontsize='x-small', fmt='%1.2f') 
    ax[0].plot(cx,cy,'w')
    plt.colorbar(cm, ax=ax[0], orientation='horizontal')
    
    ax[1].hold(True)
    ax[1].set_xlim(xlims)
    ax[1].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[1].contourf(xgrid, ygrid, Vy, np.arange(-100., 101, 5))
    cs = ax[1].contour(xgrid, ygrid, Vy, np.arange(-100, 101, 5), colors='k')
    ax[1].clabel(cs, fontsize='x-small', fmt='%1.2f') 
    plt.colorbar(cm, ax=ax[1], orientation='horizontal')
    ax[1].plot(cx,cy,'w')
    
    ax[2].hold(True)
    ax[2].set_xlim(xlims)
    ax[2].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[2].contourf(xgrid, ygrid, np.sqrt(Ux*Ux+Vy*Vy), np.arange(0., 101, 5))
    cs = ax[2].contour(xgrid, ygrid, np.sqrt(Ux*Ux+Vy*Vy), np.arange(0, 101, 5), colors='k')
    ax[2].clabel(cs, fontsize='x-small', fmt='%1.2f') 
    plt.colorbar(cm, ax=ax[2], orientation='horizontal')
    ax[2].plot(cx,cy,'w')    
    
    ax[3].hold(True)
    ax[3].set_xlim(xlims)
    ax[3].set_ylim(ylims)
    levels = np.arange(-20, 21, 2)
    cm = ax[3].contourf(xgrid, ygrid, np.sqrt(Ux*Ux+Vy*Vy), np.arange(0., 101, 5))
    ax[3].streamplot(xgrid, ygrid, Ux, Vy, density=3, color='k', linewidth=1)
    plt.colorbar(cm, ax=ax[3], orientation='horizontal')
    fig.tight_layout()
    plt.show()
